In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
## Load the trained model,scaler, and test data
model = load_model('model.h5')

## Load the scaler
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('label_encoder_Gender.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

with open('onehot_encoder_Geography.pkl', 'rb') as f:
    onehot_encoder = pickle.load(f)

In [4]:
# Example input data
input_data = {
'CreditScore': 600,
'Geography': 'France',
'Gender': 'Male',
'Age': 40,
'Tenure': 3,
'Balance': 60000,
'NumOfProducts': 2,
'HasCrCard': 1,
'IsActiveMember': 1,
'EstimatedSalary': 50000
}

In [5]:
## one-hot encode 'Geography'
geography_encoded = onehot_encoder.transform([[input_data['Geography']]]).toarray()
geography_df = pd.DataFrame(geography_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))
geography_df

c:\Users\Jervis\Desktop\DeepLearning\ANN_classification0\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
## Combine one-hot encoded columns with the rest of the input data
input_df = pd.DataFrame([input_data])
input_df = pd.concat([input_df.drop('Geography', axis=1), geography_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
## Encode categorical values
input_df['Gender'] = label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
## Scale the input data
input_scaled = scaler.transform(input_df)
input_scaled



array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [12]:
## Predict CHurn
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 109ms/step


array([[0.02905829]], dtype=float32)

In [13]:
prediction_probability = prediction[0][0]
prediction_probability

0.029058289

In [14]:
if prediction_probability > 0.5:
    print(f"The customer is likely to churn")
else:
    print(f"The customer is unlikely to churn")

The customer is unlikely to churn
